In [1]:
import numpy as np
from random import randint
import sys
sys.path.append("..")
from solvers.aco.aco_solver import AntColonyOptSolver
from problem.sudoku_manager import Sudoku, generate_boards

In [2]:
sud = Sudoku('hard', randint(0, 49))

In [3]:
print(sud.board)

[[8 0 7 0 5 0 0 0 0]
 [0 0 0 8 2 0 4 0 7]
 [0 0 0 0 0 0 0 1 3]
 [1 0 0 3 0 0 0 0 0]
 [0 0 0 0 0 6 0 2 0]
 [0 0 0 2 4 0 0 8 9]
 [2 1 0 0 0 0 0 0 0]
 [0 0 4 7 0 0 6 0 0]
 [0 0 0 4 9 0 0 3 0]]


In [4]:
print(sud.fixed_count)

25


In [5]:
print(sud.state)

{(0, 1): {2, 3, 4, 6, 9}, (0, 3): {1, 6, 9}, (0, 5): {1, 3, 4, 9}, (0, 6): {2, 9}, (0, 7): {6, 9}, (0, 8): {2, 6}, (1, 0): {3, 5, 6, 9}, (1, 1): {3, 5, 6, 9}, (1, 2): {1, 3, 5, 6, 9}, (1, 5): {1, 3, 9}, (1, 7): {5, 6, 9}, (2, 0): {4, 5, 6, 9}, (2, 1): {2, 4, 5, 6, 9}, (2, 2): {2, 5, 6, 9}, (2, 3): {6, 9}, (2, 4): {6, 7}, (2, 5): {4, 7, 9}, (2, 6): {2, 5, 8, 9}, (3, 1): {2, 4, 5, 6, 7, 8, 9}, (3, 2): {2, 5, 6, 8, 9}, (3, 4): {7, 8}, (3, 5): {5, 7, 8, 9}, (3, 6): {5, 7}, (3, 7): {4, 5, 6, 7}, (3, 8): {4, 5, 6}, (4, 0): {3, 4, 5, 7, 9}, (4, 1): {3, 4, 5, 7, 8, 9}, (4, 2): {3, 5, 8, 9}, (4, 3): {1, 5, 9}, (4, 4): {1, 7, 8}, (4, 6): {1, 3, 5, 7}, (4, 8): {1, 4, 5}, (5, 0): {3, 5, 6, 7}, (5, 1): {3, 5, 6, 7}, (5, 2): {3, 5, 6}, (5, 5): {1, 5, 7}, (5, 6): {1, 3, 5, 7}, (6, 2): {3, 5, 6, 8, 9}, (6, 3): {5, 6}, (6, 4): {3, 6, 8}, (6, 5): {3, 5, 8}, (6, 6): {5, 7, 8, 9}, (6, 7): {4, 5, 7, 9}, (6, 8): {4, 5, 8}, (7, 0): {3, 5, 9}, (7, 1): {3, 5, 8, 9}, (7, 4): {1, 3, 8}, (7, 5): {1, 2, 3, 5, 8}, 

In [6]:
solver = AntColonyOptSolver()

In [7]:
solver.solve(sudoku=sud, ants_count=64)

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7), (7, 8), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8)]
81
0
(5, 7)
1
(8, 7)
2
(1, 5)
3
(2, 6)
4
(2, 3)
5
(2, 4)
6
(7, 6)
7
(6, 5)
8
(1, 2)
9
(5, 0)
10
(1, 4)
11
(0, 0)
12
(1, 6)
13
(6, 0)
14
(1, 8)
15
(2, 1)
16
(5, 6)
17
(0, 2)
18
(6, 4)
19
(2, 5)
20
(4, 7)
21
(7, 0)
22
(5, 5)
23
(5, 2)
24
(3, 5)
25
(7, 2)
26
(4, 5)
27
(3, 1)
28
(5, 4)
29
(4, 6)
30
(0, 5)
31
(7, 3)
32
(2, 2)
33
(6, 6)
34
(3, 3)
35
(0, 4

array([[8, 3, 7, 1, 5, 4, 9, 6, 2],
       [6, 9, 1, 8, 2, 3, 4, 5, 7],
       [4, 2, 5, 9, 6, 7, 8, 1, 3],
       [1, 4, 2, 3, 8, 9, 5, 7, 6],
       [9, 8, 3, 5, 7, 6, 1, 2, 4],
       [5, 7, 6, 2, 4, 1, 3, 8, 9],
       [2, 1, 9, 6, 3, 8, 7, 4, 5],
       [3, 5, 4, 7, 1, 2, 6, 9, 8],
       [7, 6, 8, 4, 9, 5, 2, 3, 1]])